# ch8_5 dataframe preprocess part 1
이전 챕터들에서는 pandas dataframe의 기본 사용법에 대해서 알아보았습니다. 이번 챕터에서는 titanic 데이터 셋을 가지고 생존율 예측 모델을 만든다고 가정하고 데이터 전처리를 해보겠습니다. 특히 데이터 셋에 비어있는 값인 결측치를 해결하는 방법과 카테고리형 데이터를 인코딩 하는 방법에 대해서 알아보겠습니다. 

**사용하는 pandas 기능**
- df.isnull
- df.dropna
- df.fillna
- df.groupby

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("./data/titanic_train.csv")

In [4]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## 사용하지 않는 컬럼 삭제
승객의 정보로 생존율을 예측하는 모델을 만든 다고 했을 때, 승객의 id와 이름, 티켓 번호, 객실 이름은 크게 상관이 없어 보입니다. 물론 이 정보들을 활용할 수 있는 방법들도 있겠습니다만, 이 가이드에서는 날려버리겠습니다.

In [5]:
df = df.drop(["Name","Ticket","Cabin"],axis=1)

In [6]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
888,889,0,3,female,NaN,1,2,23.4500,S
889,890,1,1,male,26.0,0,0,30.0000,C


## 파생 변수 생성하기
주어진 피쳐들을 가공하여 새로운 피쳐를 만들 수 있습니다. 이렇게 만든 피쳐를 파생 변수라고 부릅니다. 타이타닉 데이터 셋에 대해서 파생 변수를 만들어보겠습니다.  

파생 변수를 만들 때에는 먼저 가설을 세우고 그에 맞는 변수를 생성하면 됩니다. 여기서는 "혼자 탑승한 승객은 가족과 함께 탑승한 승객보다 생존률이 높을 것이다"라는 가설을 세우고, 이를 잘 설명할 수 있는 파생 변수를 만들어 보겠습니다. SibSp, Parch는 각각 동승한 배우자나 형제자매의 수, 부모 또는 아이의 수입니다. 즉, SibSp와 Parch 합이 0인 사람은 혼자 탑승한 사람, 그렇지 않으면 동승자가 있는 사람이 됩니다.

In [7]:
df["FamilySize"] = df["SibSp"] +df["Parch"]

In [8]:
df["IsAlone"] = 1

In [11]:
df.loc[df["FamilySize"]>=1,"IsAlone"] = 0

In [12]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,1,0,3,male,22.0,1,0,7.2500,S,1,0
1,2,1,1,female,38.0,1,0,71.2833,C,1,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,1
3,4,1,1,female,35.0,1,0,53.1000,S,1,0
4,5,0,3,male,35.0,0,0,8.0500,S,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S,0,1
887,888,1,1,female,19.0,0,0,30.0000,S,0,1
888,889,0,3,female,NaN,1,2,23.4500,S,3,0
889,890,1,1,male,26.0,0,0,30.0000,C,0,1


데이터 셋에 원래 주어졌던 SibSp, Parch 컬럼을 이용해서 FamilySize와 IsAlone이라는 파생 변수를 만들었습니다.

## 결측값 처리하기
데이터 베이스를 공부했을 때, titanic 데이터 셋을 테이블에 집어 넣을 때 Age 값이 빠져있어서 애먹었던 기억이 있으실 겁니다. 바로 이렇게 데이터 셋에서 일부 누락된 값들을 결측치라고 부릅니다. pandas를 이용하면 이런 결측치들을 쉽게 파악할 수 있고, 결측값을 데이터 셋에서 제외하거나 적절한 값으로 채워넣을 수도 있습니다.

### 결측치 확인
데이터 프레임이 주어졌을 때, 가장 먼저 확인해봐야 할 것은 각 컬럼별로 결측값이 얼마나 포함되었는지 입니다. df.isnull()을 사용해서 쉽게 구할 수 있습니다.

In [14]:
df["Age"]

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [16]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
FamilySize       0
IsAlone          0
dtype: int64

실제로 확인해본 결과, Age, Embarked 컬럼에 결측값을 발견했습니다. 본격적으로 예측 모델을 만들기 전에, 이 결측값들을 미리 해결해야 합니다.

### 결측값 처리 1. 날려버리기
결측값을 처리하는 가장 손쉬운 방법은 결측값이 하나라도 포함된 행이나 열을 날려버리는 것입니다. Age나 Embarked가 비어있는 행만 따로 날려보도록 하겠습니다. 이 때, df.drop, df.dropna 함수를 사용하면 됩니다.

In [17]:
df["Age"].tolist()[:10]

[22.0, 38.0, 26.0, 35.0, 35.0, nan, 54.0, 2.0, 27.0, 14.0]

In [19]:
df["Age"].fillna(100).tolist()[:10]

[22.0, 38.0, 26.0, 35.0, 35.0, 100.0, 54.0, 2.0, 27.0, 14.0]

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

결측값이 있는 행을 모조리 날려버려서 결측치 없는 데이터 프레임을 얻을 수 있었습니다.

### 결측값 처리 2. 임의의 값을 채워넣기
가뜩이나 없는 데이터를 Age 값 하나 빠졌다고 100개가 넘는 행을 날려버리는 것은 무척이나 아깝습니다. 그럴듯한 값으로 채워넣으면 충분히 살릴 수 있지 않을까요?

series.fillna를 사용하면 nan 값을 특정 값으로 채워넣을 수 있습니다. 한번 비어있는 Age 값을 100으로 채워보겠습니다.

비어있던 Age 값이 100으로 채워졌습니다. 비어있는 값을 채워넣더라도 그럴싸한 값으로 채워넣으면 어떨까요? 예를들어 100으로 채워넣기보단 전체 승객의 평균 연령으로 채워넣는 것이 더 좋아보입니다. dataframe의 groupby를 사용하면 쉽게 그룹별 평균을 계산할 수 있습니다.

In [23]:
df.groupby("Sex")["Age"].mean()

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

남성 평균 연령은 27세, 여성 평균 연령은 30세로 집계되었습니다. 이를 좌석 등급별로 한번 더 나눠보면 더 정확하지 않을까요? 한번 해보겠습니다.

In [25]:
df.groupby(["Sex","Pclass"])["Age"].mean()

Sex     Pclass
female  1         34.611765
        2         28.722973
        3         21.750000
male    1         41.281386
        2         30.740707
        3         26.507589
Name: Age, dtype: float64

In [27]:
mean_age_dict = df.groupby(["Sex","Pclass"])["Age"].mean().to_dict()
mean_age_dict

{('female', 1): 34.61176470588235,
 ('female', 2): 28.722972972972972,
 ('female', 3): 21.75,
 ('male', 1): 41.28138613861386,
 ('male', 2): 30.74070707070707,
 ('male', 3): 26.507588932806325}

더 세분화 할수도 있겠으나, 그 이상 잘게 쪼개면 표본이 너무 적어지므로, 성별 + 좌석 등급별 평균 연령으로 결측치를 채워넣어 보겠습니다. 나중에 test 데이터프레임에도 같은 값을 채워넣어야 하므로, 각 성별 + 좌석 등급 그룹에 고정값을 할당하는 방식으로 채워넣어 보겠습니다.  

In [36]:
def fill_group(group):
    sex,pclass=group.iloc[0][["Sex","Pclass"]]
    print(sex,pclass)
    fill_value = mean_age_dict[(sex,pclass)]
    group["Age"] = group["Age"].fillna(fill_value)
    return group

In [37]:
df = df.groupby(["Sex","Pclass"]).apply(fill_group)

female 1
female 2
female 3
male 1
male 2
male 3


C:\Users\WOODLAC\AppData\Local\Temp\ipykernel_17676\419592874.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = df.groupby(["Sex","Pclass"]).apply(fill_group)


Age 결측치를 모두 채워넣어 봤습니다. Embarked 결측치는 가장 많이 탑승한 승선지로 채워넣겠습니다. 

In [38]:
df["Embarked"] = df["Embarked"].fillna("S")

In [39]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
FamilySize     0
IsAlone        0
dtype: int64

In [46]:
df["Sex"] = lable_encoder.fit_transform(df["Sex"])
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,1,0,3,1,22.00,1,0,7.2500,2,1,0
1,2,1,1,0,38.00,1,0,71.2833,0,1,0
2,3,1,3,0,26.00,0,0,7.9250,2,0,1
3,4,1,1,0,35.00,1,0,53.1000,2,1,0
4,5,0,3,1,35.00,0,0,8.0500,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.00,0,0,13.0000,2,0,1
887,888,1,1,0,19.00,0,0,30.0000,2,0,1
888,889,0,3,0,21.75,1,2,23.4500,2,3,0
889,890,1,1,1,26.00,0,0,30.0000,0,0,1


지금까지 결측치를 다루는 대표적인 2가지 방법을 알아보았습니다. 이 외에도 결측치를 그냥 아예 하나의 값으로 두는 방법, 머신러닝 모델로 최적의 결측치를 예측하는 방법 등이 있습니다. 정답이 정해져 있는 것이 아니고, 풀어야 하는 문제와 데이터 셋의 특성에 따라서 적절한 방식을 선택하면 됩니다.

## 카테고리형 데이터 인코딩
사용 안하는 컬럼을 제거하고, 결측값을 메꾼 데이터프레임을 얻었습니다.

In [40]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,1,0,3,male,22.0,1,0,7.2500,S,1,0
1,2,1,1,female,38.0,1,0,71.2833,C,1,0
2,3,1,3,female,26.0,0,0,7.9250,S,0,1
3,4,1,1,female,35.0,1,0,53.1000,S,1,0
4,5,0,3,male,35.0,0,0,8.0500,S,0,1


### Label Encoding
여기서 Age, Fare, SibSp, Parch는 특정 값을 나타내는 숫자이지만 Pclass는 종류를 나타내는 숫자입니다. 즉, 1등석 인지 2등석인지 3등석인지를 범주를 나타내는 숫자입니다. 비슷하게 Sex, Embarked 컬럼도 모두 범주를 나타내는 컬럼들이지만 문자열 값들을 갖습니다. 예측 모델을 만들기 전에, 이 범주형 데이터들을 숫자로 적절히 변환해주어야 합니다. 이를 레이블 인코딩이라 부릅니다.

먼저 Embarked 컬럼에 어떤 값들이 채워져 있는지 확인하겠습니다.

In [41]:
df["Embarked"].unique()

array(['S', 'C', 'Q'], dtype=object)

S, C, Q 세 가지 값이 있네요. scikit-learn의 LabelEncoder를 사용하면 편하게 범주형 데이터를 숫자로 변환할 수 있습니다.

In [42]:
!pip install scikit-learn


                                              0.0/8.3 MB ? eta -:--:--
     ------                                   1.3/8.3 MB 27.7 MB/s eta 0:00:01
     ----------------                         3.3/8.3 MB 35.3 MB/s eta 0:00:01
     ----------------------------             5.9/8.3 MB 41.8 MB/s eta 0:00:01
     ---------------------------------------  8.1/8.3 MB 43.3 MB/s eta 0:00:01
     ---------------------------------------- 8.3/8.3 MB 40.6 MB/s eta 0:00:00
                                              0.0/42.2 MB ? eta -:--:--
     -                                        1.9/42.2 MB 61.1 MB/s eta 0:00:01
     ---                                      4.2/42.2 MB 53.4 MB/s eta 0:00:01
     ------                                   6.5/42.2 MB 51.8 MB/s eta 0:00:01
     --------                                 8.8/42.2 MB 51.1 MB/s eta 0:00:01
     ----------                              11.0/42.2 MB 50.4 MB/s eta 0:00:01
     ------------                            13.3/42.2 MB 50.1

In [43]:
from sklearn.preprocessing import LabelEncoder
lable_encoder = LabelEncoder()

In [47]:
df["Embarked"] = lable_encoder.fit_transform(df["Embarked"])
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,1,0,3,1,22.00,1,0,7.2500,2,1,0
1,2,1,1,0,38.00,1,0,71.2833,0,1,0
2,3,1,3,0,26.00,0,0,7.9250,2,0,1
3,4,1,1,0,35.00,1,0,53.1000,2,1,0
4,5,0,3,1,35.00,0,0,8.0500,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.00,0,0,13.0000,2,0,1
887,888,1,1,0,19.00,0,0,30.0000,2,0,1
888,889,0,3,0,21.75,1,2,23.4500,2,3,0
889,890,1,1,1,26.00,0,0,30.0000,0,0,1


### One Hot Encoding
앞서 범주형 데이터 컬럼들을 숫자로 변환했습니다. 그런데 Embarked 컬럼을 보면 0, 1, 2 이 숫자들은 사실은 특정 승선지를 나타내는 숫자들입니다. 때문에 값들 간에 대소 관계는 없지만 숫자를 사용했기 때문에 혼동이 옵니다. 

이를 피하기 위해서 각 범주별로 컬럼을 쪼개고, 해당 범주에 속하는지 여부만 0과 1로 표현하도록 변환합니다. 이를 one-hot encoding이라고 부르며 pandas의 dummies 함수를 이용하여 쉽게 구현할 수 있습니다.

In [51]:
embarked_dummies = pd.get_dummies(df["Embarked"], prefix="Embarked")

In [52]:
df = pd.concat([df,embarked_dummies], axis=1)
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Embarked_0,Embarked_1,Embarked_2
0,1,0,3,1,22.00,1,0,7.2500,2,1,0,0,0,1
1,2,1,1,0,38.00,1,0,71.2833,0,1,0,1,0,0
2,3,1,3,0,26.00,0,0,7.9250,2,0,1,0,0,1
3,4,1,1,0,35.00,1,0,53.1000,2,1,0,0,0,1
4,5,0,3,1,35.00,0,0,8.0500,2,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.00,0,0,13.0000,2,0,1,0,0,1
887,888,1,1,0,19.00,0,0,30.0000,2,0,1,0,0,1
888,889,0,3,0,21.75,1,2,23.4500,2,3,0,0,0,1
889,890,1,1,1,26.00,0,0,30.0000,0,0,1,1,0,0


one-hot encoding을 적용한 컬럼들을 기존 데이터 프레임에 합쳐줍니다. 이 때, df.concat을 사용합니다. 행을 붙이는 것이 아닌, 컬럼을 붙이는 것이므로 axis=1로 설정해줍니다.

In [53]:
Pclass_dummies = pd.get_dummies(df["Pclass"], prefix="Pclass")
df = pd.concat([df,Pclass_dummies], axis=1)
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Embarked_0,Embarked_1,Embarked_2,Pclass_1,Pclass_2,Pclass_3
0,1,0,3,1,22.00,1,0,7.2500,2,1,0,0,0,1,0,0,1
1,2,1,1,0,38.00,1,0,71.2833,0,1,0,1,0,0,1,0,0
2,3,1,3,0,26.00,0,0,7.9250,2,0,1,0,0,1,0,0,1
3,4,1,1,0,35.00,1,0,53.1000,2,1,0,0,0,1,1,0,0
4,5,0,3,1,35.00,0,0,8.0500,2,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.00,0,0,13.0000,2,0,1,0,0,1,0,1,0
887,888,1,1,0,19.00,0,0,30.0000,2,0,1,0,0,1,1,0,0
888,889,0,3,0,21.75,1,2,23.4500,2,3,0,0,0,1,0,0,1
889,890,1,1,1,26.00,0,0,30.0000,0,0,1,1,0,0,1,0,0


In [54]:
df = df.drop(["Pclass","Embarked"], axis=1)
df

,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,FamilySize,IsAlone,Embarked_0,Embarked_1,Embarked_2,Pclass_1,Pclass_2,Pclass_3
0,1,0,1,22.00,1,0,7.2500,1,0,0,0,1,0,0,1
1,2,1,0,38.00,1,0,71.2833,1,0,1,0,0,1,0,0
2,3,1,0,26.00,0,0,7.9250,0,1,0,0,1,0,0,1
3,4,1,0,35.00,1,0,53.1000,1,0,0,0,1,1,0,0
4,5,0,1,35.00,0,0,8.0500,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,1,27.00,0,0,13.0000,0,1,0,0,1,0,1,0
887,888,1,0,19.00,0,0,30.0000,0,1,0,0,1,1,0,0
888,889,0,0,21.75,1,2,23.4500,3,0,0,0,1,0,0,1
889,890,1,1,26.00,0,0,30.0000,0,1,1,0,0,1,0,0


Pclass 컬럼도 one-hot encoding을 적용해보겠습니다.

### 전처리를 마친 컬럼 제거
one-hot encoding을 적용한 Pclass, Embarked 컬럼은 제거

## 결과 저장

전처리 과정이 아직 남아있습니다만, 이번 챕터가 너무 길어져서 끊고 가겠습니다. 데이터 프레임의 to_csv() 함수를 이용하면 쉽게 데이터 프레임을 저장할 수 있습니다.

In [55]:
df.to_csv("./data/tatanic_train_preprocessed.csv")

## 정리
지금까지 타이타닉 생존자 데이터 셋을 pandas data frame으로 만들어서 여러 전처리를 해보았습니다. 사용하지 않는 컬럼을 삭제하고, 파생 변수를 만들고, 결측치를 해결하고, 변수들을 숫자로 인코딩 하는 등의 작업이 포함되었습니다. 이 모든 과정을 묶어서 피쳐 엔지니어링이라고 부릅니다.

워낙 유명한 예제여서 찾아보시면 다양한 예제들과 기법들이 나올 것입니다. 정해진 정답은 없다는걸 기억하고, 적절해 보이는 기법들을 취사 선택해서 이용하면 됩니다.